In [ ]:
# import logging; logging.basicConfig(level=logging.DEBUG)
from __future__ import absolute_import, unicode_literals, print_function

import dropbot as db
import pandas as pd

In [ ]:
try:
    proxy.terminate()
except NameError:
    pass

proxy = db.SerialProxy(ignore=True)
signal = proxy.signals.signal('capacitance-updated')

proxy.update_state(hv_output_selected=True, hv_output_enabled=True, voltage=100, frequency=10e3)
proxy.state

In [ ]:
class Plot(object):
    def __init__(self, *args, **kwargs):
        self._args = args
        self._kwargs = kwargs
    
    def display(self):
        fig = plt.figure(*self._args, **self._kwargs)
        ax = fig.add_subplot(111)
        self.ax = ax
        ax.patch.set_alpha(0.1)
        plt.ion()

        fig.show()
        fig.canvas.draw()

        self.line = ax.plot([], [])[0]
        siC_formatter = FuncFormatter(lambda x, *args: si.si_format(x) + 'F')
        ax.yaxis.set_major_formatter(siC_formatter)
        return ax

    def update(self, data_series):
        self.line.set_data((data_series.index,
                            data_series.values))
        self.ax.relim()
        self.ax.autoscale()
        fig = self.ax.get_figure()
        fig.canvas.draw()
        fig.tight_layout()

In [ ]:
class DataLogger(object):
    def __init__(self, n, value_key, index_key):
        self.values = []
        self.index = []
        self.value_key = value_key
        self.index_key = index_key
        self.n = n

    def save(self, event):
        self.values.append(event.get(self.value_key))
        self.index.append(event.get(self.index_key))

        for array in (self.values, self.index):
            if len(array) > self.n:
                array.pop(0)
                
    def to_series(self):
        return pd.Series(self.values, index=self.index)

In [ ]:
%matplotlib notebook
from __future__ import absolute_import, print_function, unicode_literals

from matplotlib.ticker import FuncFormatter
import functools as ft
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import si_prefix as si

import threading


class PlotController(object):
    def __init__(self, proxy, n, refresh_interval=.25, update_interval=None):
        self.plot_stop = threading.Event()
        self.refresh_stopped = threading.Event()
        self.plot = Plot()
        self.data_logger = DataLogger(n=n, value_key='new_value', index_key='end')
        self.proxy = proxy
        self.proxy.signals.signal('capacitance-updated').connect(self.data_logger.save)
        self._refresh_interval = refresh_interval
        if update_interval is None:
            self._update_interval = refresh_interval
        else:
            self._update_interval = update_interval
        
    @property
    def update_interval(self):
        return self._update_interval
    
    @update_interval.setter
    def update_interval(self, value):
        self._update_interval = value
        self.stop()
        self.start()
    
    @property
    def refresh_interval(self):
        return self._refresh_interval
    
    @refresh_interval.setter
    def refresh_interval(self, value):
        self._refresh_interval = value
        self.stop()
        self.start()
    
    def display(self):
        self.start_button = ipywidgets.Button(description='Start')
        self.stop_button = ipywidgets.Button(description='Stop')

        N = int(proxy.number_of_channels)
        self.channels = [ipywidgets.SelectMultiple(options=range(40 * i, 40 * (i + 1))[:N], rows=40)
                         for i in range(int(np.ceil(N / 40.)))]
        for channels_i in self.channels:
            channels_i.observe(self.actuate_selected, names=['value'], type='change')

        self.stop_button.disabled = True
        self.start_button.on_click(self.start)
        self.stop_button.on_click(self.stop)
        self.widgets = ipywidgets.HBox([self.start_button, self.stop_button] + self.channels)
        return self.plot.display(), self.widgets
        
    def actuate_selected(self, event):
        state_of_channels = proxy.state_of_channels
        state_of_channels[list(event['owner'].options)] = 0
        state_of_channels[list(event['new'])] = 1
        proxy.state_of_channels = state_of_channels

    def start(self, *args):
        self.plot_stop.clear()
        self.refresh_stopped.clear()

        proxy.update_state(capacitance_update_interval_ms=int(self.update_interval * 1e3))
        self.start_button.disabled = True
        self.stop_button.disabled = False
        
        def _refresh_plot(refresh_interval):
            while not self.plot_stop.wait(refresh_interval):
                self.plot.update(self.data_logger.to_series())
            self.refresh_stopped.set()

        self.thread = threading.Thread(target=_refresh_plot, args=(self.refresh_interval, ))
        self.thread.daemon = True
        self.thread.start()

    def stop(self, *args):
        self.plot_stop.set()
        self.refresh_stopped.wait(5)
        
        self.proxy.update_state(capacitance_update_interval_ms=0)
        self.start_button.disabled = False
        self.stop_button.disabled = True

In [ ]:
controller = PlotController(proxy, 100, update_interval=.5)
fig, controls = controller.display()
controls